In [3]:
import numpy as np
import torch
import sys
sys.path.append('../')
from voting_games.werewolf_env_v0 import raw_env, Roles
import random
from tqdm import tqdm

/root/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class Agent(torch.nn.Module):
    def __init__(self, num_actions, obs_size):
        super().__init__()

        self.critic = torch.nn.Sequential(
            self._layer_init(torch.nn.Linear(obs_size, 64)),
            torch.nn.Tanh(),
            self._layer_init(torch.nn.Linear(64,64)),
            torch.nn.Tanh(),
            self._layer_init(torch.nn.Linear(64,1), std=1.0),
        )

        self.actor = torch.nn.Sequential(
            self._layer_init(torch.nn.Linear(obs_size, 64)),
            torch.nn.Tanh(),
            self._layer_init(torch.nn.Linear(64,64)),
            torch.nn.Tanh(),
            self._layer_init(torch.nn.Linear(64, num_actions), std=0.01),
        )
    
    def get_value(self, x):
        return self.critic(x)
    
    def get_action_and_value(self, x, action=None):
        # could call the network each time, with a different integer for each player?  get approvals that way
        logits = self.actor(x)
        probs = torch.distributions.categorical.Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy(), self.critic(x)

    def _layer_init(self, layer, std=np.sqrt(2), bias_const=0.0):
        torch.nn.init.orthogonal_(layer.weight, std)
        torch.nn.init.constant_(layer.bias, bias_const)
        return layer

In [ ]:
def get_killed(lists):
    

In [21]:
p1 = np.array([1,0,0,0])
p2 = np.array([1,1,0,0])
p3 = np.array([0,1,1,0])

dead_players = [0]

votes = p1 + p2 + p3
max_indices = np.where(votes == max(votes))[0]
dead_vote_flag = False
tie_vote_flag = False

# if we have a tie, keep the living players and randomly choose between them, report back a tie
living_selections = [player for player in max_indices if player not in dead_players]

if len(max_indices) > 1:
    tie_vote_flag = True


if len(living_selections) < len(max_indices):
    dead_vote_flag = True

# If we have any living selections, lets sampple one
if len(living_selections) > 0:
    return random.sample(living_selections), dead_vote_flag, tie_vote_flag

# keep going down the chain
for next_best in np.argsort(votes)[::-1][len(max_indices):]:
    if next_best not in dead_players:
        return next_best, dead_vote_flag, tie_vote_flag




# now if they are all dead, we have to move on to the next 
#np.argsort(votes)[::-1]

[0 1]
[True, False]
[2 3]


array([1, 0, 2, 3])

In [18]:
sum([True,True])

2